In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from data_processing_service import DataProcessingService

# Initialize DataProcessingService
data_processor = DataProcessingService(seq_length=10, pred_window=1, batch_size=10)
data = data_processor.get_sample_data(length=50)

# Normalize the data
data_normalized, scaler = data_processor.normalize_data(data)

# Create sequences
x_data, y_data = data_processor.create_sequences(data_normalized)

# Split data into training and test sets
x_train, x_test, y_train, y_test = data_processor.split_data(x_data, y_data, train_ratio=0.8)

# Convert data to PyTorch tensors
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [2]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

torch.Size([32, 10, 1]) torch.Size([32, 1, 1]) torch.Size([8, 10, 1]) torch.Size([8, 1, 1])


In [4]:
# Define a simple neural network model
from model_service import LinRegNN

# Initialize the model
input_size = data_processor.seq_length
hidden_size = 10
output_size = data_processor.pred_window
model = LinRegNN(input_size, output_size)


# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
x_train_rs,x_test_rs= model.reshape_input(x_train),model.reshape_input(x_test)
epochs = 10
model.train_loop(model, criterion, optimizer, x_train_rs, y_train, epochs)
# Eval
model.evaluation(model, x_test_rs, y_test, criterion)


Epoch [2/10], Loss: 0.4505
Epoch [4/10], Loss: 0.2732
Epoch [6/10], Loss: 0.1563
Epoch [8/10], Loss: 0.0869
Epoch [10/10], Loss: 0.0491
Test Loss: 0.1162
